# Create a Convolutional Neural Network using Python and libraries

The goal of this project is to create a CNN and learn it to recognize faces in photos.
We will use open-sources pictures libraries to train our model.
We will implement the following steps:
1. Import all needed libraries
2. Import images dataset, normalize it and shape it to fit the NN
3. Initialize the NN 
4. Train the NN with train dataset and check with validation dataset
5. Test the NN on test dataset

## Initialization

### 1.Import all needed libraries

We begin by initializing our programing environnement, importing every library needed.


In [16]:
!pip install torchvision

In [15]:
import matplotlib.pyplot as plt
import torch
import numpy as np
import pandas as pd
import seaborn as sb
import random
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import torch.optim as optim

In [4]:
!pip install -U skorch

Requirement already up-to-date: skorch in c:\users\maxime dardy\.conda\envs\ml\lib\site-packages (0.6.0)


In [5]:
from skorch import NeuralNetBinaryClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split

### 2. Import images dataset and shape it

Importing the dataset and normalizing it to fit into the NN

In [ ]:
transform = transforms.Compose(
    [transforms.Resize((36,36)),
        transforms.Grayscale(),
        transforms.ToTensor(),
        transforms.Normalize((0.5,), (0.5,))
        # transforms.Normalize((0.5,0.5,0.5), (0.5,0.5,0.5))
    ])

trainsetTotal = torchvision.datasets.ImageFolder(root='./start_deep/train_images',transform=transform)

trainloader = torch.utils.data.DataLoader(trainsetTotal, batch_size=64,
                                          shuffle=True)



testset = torchvision.datasets.ImageFolder(root='./start_deep/test_images_custom',transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=64,
                                         shuffle=False)
classes=('non-visage','visage')

print images

In [ ]:
def imshow(img):
    img = img / 2 + 0.5     # unnormalize
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()


# get some random training images
dataiter = iter(trainloader)
data = dataiter.next()

# show images
imshow(torchvision.utils.make_grid(data[0][0:15], nrow = 16))
# print labels
print(' '.join('%5s' % classes[data[1][j]] for j in range(15)))

### 2. Initialize the NN

we create our network and instanciate it using the torch.nn library

In [ ]:
class CNN(nn.Module):
    
    #define the architecture of the NN
    def __init__(self):
        super(CNN,self).__init__()
        # 1 input chanel as images are in gray nuance
        # 6 output channels
        # 5x5 square convolution
        self.conv1 = nn.Conv2d(1,6,5)
        self.pool1 = nn.MaxPool2d(2,2)
        self.conv2 = nn.Conv2d(6,20,5)
        self.conv3 = nn.Conv2d(20,30,5)
        #self.pool2 = nn.MaxPool2d(2,2)
        #ad affine operation y = Wx + b
        #correspond to the differents layers of the NN
        # with W = weight of the neurones and B = bias
        #self.fc1 = nn.Linear(20*5*5,200)
        self.fc1 = nn.Linear(30*8*8,200)
        self.fc2 = nn.Linear(200,80)
        self.fc3 = nn.Linear(80,2)
    
    #define the feed-forward algorithm for the NN
    
    def forward(self, x):
        x = self.pool1(F.relu(self.conv1(x)))
        #x = self.pool2(F.relu(self.conv2(x)))
        x = (F.relu(self.conv2(x)))
        x = (F.relu(self.conv3(x)))
        #x = x.view(-1, 20*5*5)
        x = x.view(-1, 30*8*8)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.sigmoid(self.fc3(x))
        return x
        
cnn = CNN()

We create the torch wrapper in order to use the skitlearn functions

In [39]:
net = NeuralNetBinaryClassifier(
    module = CNN,
    #criterion= nn.CrossEntropyLoss,
    lr = 0.001,
    optimizer = optim.SGD,
    threshold=0.5,
    max_epochs = 4,
    batch_size =128,
    warm_start=True 
)

NameError: name 'NeuralNetBinaryClassifier' is not defined

We now create:
* the criterion which will calculate the Loss function
    in this case we use CrossEntropyLoss: a combination of Softmax and Loss fct
* the optimizer which the algorithm used to compute the weight (grad desc, Adam...)
    in this case we use Stochastical Gradient Descent

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(cnn.parameters(), lr=0.001, momentum=0.9)

### 4. Train and validate the NN

Train and validate the NN using skitlearn with cross validation

In [36]:
#trainingFeatures = trainingFeatures.view(-1,1)
net.fit(trainingFeatures,trainingLabels)

NameError: name 'net' is not defined

In [7]:
for epoch in range(6): # loop over the dataset multiple times
    
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        #get the inputs; data is a list of [inputs, labels]
        inputs, labels = data
        
       # imshow(torchvision.utils.make_grid(inputs[0:10]))
       # print(' '.join('%5s' % classes[labels[j]] for j in range(10)))
        
        #zero the parameter gradients
        optimizer.zero_grad()
        
        #forward + backward prop + optimizer
        outputs = cnn(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        #print statistics
        running_loss += loss.item()
        if i %100 == 99: #print every 100 mini-batches
            print('[%d, %5d loss: %.3f]' %
                 (epoch +1, i +1, running_loss / 100))
            running_loss = 0.0
            
print('Finished Training')

[1,   100 loss: 0.641]
[1,   200 loss: 0.638]
[1,   300 loss: 0.636]
[1,   400 loss: 0.634]
[1,   500 loss: 0.629]
[1,   600 loss: 0.623]
[1,   700 loss: 0.620]
[1,   800 loss: 0.619]
[1,   900 loss: 0.605]
[1,  1000 loss: 0.600]
[1,  1100 loss: 0.601]
[1,  1200 loss: 0.600]
[1,  1300 loss: 0.585]
[1,  1400 loss: 0.587]
[2,   100 loss: 0.583]
[2,   200 loss: 0.577]
[2,   300 loss: 0.576]
[2,   400 loss: 0.571]
[2,   500 loss: 0.567]
[2,   600 loss: 0.564]
[2,   700 loss: 0.560]
[2,   800 loss: 0.564]
[2,   900 loss: 0.549]
[2,  1000 loss: 0.536]
[2,  1100 loss: 0.507]
[2,  1200 loss: 0.478]
[2,  1300 loss: 0.460]
[2,  1400 loss: 0.456]
[3,   100 loss: 0.446]
[3,   200 loss: 0.438]
[3,   300 loss: 0.430]
[3,   400 loss: 0.435]
[3,   500 loss: 0.429]
[3,   600 loss: 0.415]
[3,   700 loss: 0.418]
[3,   800 loss: 0.417]
[3,   900 loss: 0.406]
[3,  1000 loss: 0.404]
[3,  1100 loss: 0.402]
[3,  1200 loss: 0.403]
[3,  1300 loss: 0.399]
[3,  1400 loss: 0.396]
[4,   100 loss: 0.390]
[4,   200 l

We save our trained model

In [8]:
PATH = './CNN_Normalized.pth'
torch.save(cnn.state_dict(), PATH)

In [9]:
cnn = CNN()
PATH = './CNN_Normalized.pth'
cnn.load_state_dict(torch.load(PATH))

### 5. Test the NN on test dataset

In [36]:
correct = 0
total = 0
seuil = 0.9

with torch.no_grad():
    for data in testloader:
        imagesTest, labelsTest = data
        outputsTest = cnn(imagesTest)[0][1]
        predicted = outputsTest >= seuil
        total += labelsTest.size(0)
        correct += (predicted == labelsTest).sum().item()

print('Accuracy of the network on the 10000 test images: %d %%' % (
    100 * correct / total))


RuntimeError: Expected object of scalar type Byte but got scalar type Long for argument #2 'other'

### 6. Detect the faces in general images

In [12]:
!pip install imutils

In [26]:
import argparse
import cv2
import matplotlib.pyplot as plt
import time
import PIL
import imutils
from PIL import Image
from matplotlib import cm

transformer = transforms.Compose([transforms.Resize(36),
                                      transforms.Grayscale(),
                                      transforms.ToTensor(),
                                      transforms.Normalize((0.5,), (0.5,))
                                     ])

def pyramid(image, scale=1.2, minSize=(36,36)):
    yield image

    while True:
        #compute the new dimensions and resize
        w = int(image.shape[1] / scale)
        image = imutils.resize(image, width = w)

        #if image is small enough stop
        if image.shape[0] < minSize[1] or image.shape[1] < minSize[0]:
            break

        yield image


def sliding_window(image, stepSize, windowSize):
    #sliding a window across the image
    for y in range(0, image.shape[0], stepSize):
        for x in range (0, image.shape[1], stepSize):
            #yield the current window
            yield (x, y, image[y:y + windowSize[1], x:x + windowSize[0]])
            
def resized_roi(x, y, width, height, factor):
    new_width = width * factor
    new_height = height * factor
    new_x = x - (new_width-width)/2
    new_y = y - (new_height-height)/2
    
    return([round(new_x), round(new_y), round(new_width), round(new_height)])



            
# image = pil_image = PIL.Image.open('image.jpg')
# plt.imshow(image)
# plt.show()


image = cv2.imread("turbo_renoi.jpg")
clone = image.copy()
# image = cv2.normalize(image, None, alpha=0, beta=1, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_32F)

# You may need to convert the color.

(winW, winH) = (36, 36)

#loop over the pyramid
for (i, resized) in enumerate(pyramid(image)):
    # loop over the sliding window for each layer of the pyramid
	for (x, y, window) in sliding_window(resized, stepSize=15, windowSize=(winW, winH)):
		# if the window does not meet our desired window size, ignore it
		if window.shape[0] != winH or window.shape[1] != winW:
			continue

		ROI = resized[y:y+winH, x:x+winW]
		ROI = cv2.cvtColor(ROI, cv2.COLOR_BGR2RGB)
		PIL_ROI = Image.fromarray(ROI)
		ROI_tensor = transformer(PIL_ROI).float()
		ROI_tensor = ROI_tensor.unsqueeze_(0)
		isFace = cnn(ROI_tensor)[0][1]
		print(isFace)
        
# 		if (((isFace == isFace.max()).nonzero())[0][1] == 1) :
		if (isFace >= 0.9995) :
			ROI_to_draw = resized_roi(x*1.2**i,y*1.2**i,winW,winH, 1.2**i)
			cv2.imshow("ROI",ROI)
			if i>=0 : 
				cv2.rectangle(clone, (ROI_to_draw[0], ROI_to_draw[1]), (ROI_to_draw[0] + ROI_to_draw[2], ROI_to_draw[1] + ROI_to_draw[3]), (0, 255, 0), 2)
			cv2.imshow("Window", clone)
			cv2.waitKey(1)
         
# 		since we do not have a classifier, we'll just draw the window
# 		clone = resized.copy()
# 		cv2.rectangle(clone, (x, y), (x + winW, y + winH), (0, 255, 0), 2)
# 		cv2.imshow("Window", clone)
# 		cv2.waitKey(1)

tensor(0.0097, grad_fn=<SelectBackward>)
tensor(0.0367, grad_fn=<SelectBackward>)
tensor(0.0239, grad_fn=<SelectBackward>)
tensor(0.0544, grad_fn=<SelectBackward>)
tensor(0.0543, grad_fn=<SelectBackward>)
tensor(0.0031, grad_fn=<SelectBackward>)
tensor(0.0034, grad_fn=<SelectBackward>)
tensor(0.3425, grad_fn=<SelectBackward>)
tensor(0.0406, grad_fn=<SelectBackward>)
tensor(0.0450, grad_fn=<SelectBackward>)
tensor(0.4986, grad_fn=<SelectBackward>)
tensor(0.0206, grad_fn=<SelectBackward>)
tensor(0.0017, grad_fn=<SelectBackward>)
tensor(0.0030, grad_fn=<SelectBackward>)
tensor(0.0082, grad_fn=<SelectBackward>)
tensor(0.0133, grad_fn=<SelectBackward>)
tensor(0.1433, grad_fn=<SelectBackward>)
tensor(0.4387, grad_fn=<SelectBackward>)
tensor(0.0068, grad_fn=<SelectBackward>)
tensor(0.0015, grad_fn=<SelectBackward>)
tensor(0.0046, grad_fn=<SelectBackward>)
tensor(0.0054, grad_fn=<SelectBackward>)
tensor(0.0088, grad_fn=<SelectBackward>)
tensor(0.0049, grad_fn=<SelectBackward>)
tensor(0.0148, g

tensor(0.5128, grad_fn=<SelectBackward>)
tensor(0.2398, grad_fn=<SelectBackward>)
tensor(0.0241, grad_fn=<SelectBackward>)
tensor(0.1367, grad_fn=<SelectBackward>)
tensor(0.0674, grad_fn=<SelectBackward>)
tensor(0.1426, grad_fn=<SelectBackward>)
tensor(0.0996, grad_fn=<SelectBackward>)
tensor(0.0901, grad_fn=<SelectBackward>)
tensor(0.0496, grad_fn=<SelectBackward>)
tensor(0.0795, grad_fn=<SelectBackward>)
tensor(0.3402, grad_fn=<SelectBackward>)
tensor(0.0438, grad_fn=<SelectBackward>)
tensor(0.2986, grad_fn=<SelectBackward>)
tensor(0.0169, grad_fn=<SelectBackward>)
tensor(0.0022, grad_fn=<SelectBackward>)
tensor(0.0039, grad_fn=<SelectBackward>)
tensor(0.0002, grad_fn=<SelectBackward>)
tensor(0.0121, grad_fn=<SelectBackward>)
tensor(0.0010, grad_fn=<SelectBackward>)
tensor(0.0037, grad_fn=<SelectBackward>)
tensor(0.0080, grad_fn=<SelectBackward>)
tensor(0.0761, grad_fn=<SelectBackward>)
tensor(0.1754, grad_fn=<SelectBackward>)
tensor(0.1387, grad_fn=<SelectBackward>)
tensor(0.0143, g

tensor(0.4276, grad_fn=<SelectBackward>)
tensor(0.3423, grad_fn=<SelectBackward>)
tensor(0.4440, grad_fn=<SelectBackward>)
tensor(0.3079, grad_fn=<SelectBackward>)
tensor(0.3699, grad_fn=<SelectBackward>)
tensor(0.1542, grad_fn=<SelectBackward>)
tensor(0.2071, grad_fn=<SelectBackward>)
tensor(0.4370, grad_fn=<SelectBackward>)
tensor(0.6886, grad_fn=<SelectBackward>)
tensor(0.8766, grad_fn=<SelectBackward>)
tensor(0.9667, grad_fn=<SelectBackward>)
tensor(0.8854, grad_fn=<SelectBackward>)
tensor(0.8328, grad_fn=<SelectBackward>)
tensor(0.6000, grad_fn=<SelectBackward>)
tensor(0.1664, grad_fn=<SelectBackward>)
tensor(0.0894, grad_fn=<SelectBackward>)
tensor(0.1158, grad_fn=<SelectBackward>)
tensor(0.1028, grad_fn=<SelectBackward>)
tensor(0.2016, grad_fn=<SelectBackward>)
tensor(0.1600, grad_fn=<SelectBackward>)
tensor(0.1391, grad_fn=<SelectBackward>)
tensor(0.1712, grad_fn=<SelectBackward>)
tensor(0.1386, grad_fn=<SelectBackward>)
tensor(0.2845, grad_fn=<SelectBackward>)
tensor(0.7320, g

tensor(0.8309, grad_fn=<SelectBackward>)
tensor(0.0877, grad_fn=<SelectBackward>)
tensor(0.0017, grad_fn=<SelectBackward>)
tensor(0.0049, grad_fn=<SelectBackward>)
tensor(0.0013, grad_fn=<SelectBackward>)
tensor(0.0012, grad_fn=<SelectBackward>)
tensor(0.0016, grad_fn=<SelectBackward>)
tensor(0.0070, grad_fn=<SelectBackward>)
tensor(0.0974, grad_fn=<SelectBackward>)
tensor(0.0055, grad_fn=<SelectBackward>)
tensor(0.0241, grad_fn=<SelectBackward>)
tensor(0.8189, grad_fn=<SelectBackward>)
tensor(0.6060, grad_fn=<SelectBackward>)
tensor(0.3607, grad_fn=<SelectBackward>)
tensor(0.2817, grad_fn=<SelectBackward>)
tensor(0.4506, grad_fn=<SelectBackward>)
tensor(0.0846, grad_fn=<SelectBackward>)
tensor(0.0206, grad_fn=<SelectBackward>)
tensor(0.0497, grad_fn=<SelectBackward>)
tensor(0.0586, grad_fn=<SelectBackward>)
tensor(0.3608, grad_fn=<SelectBackward>)
tensor(0.2547, grad_fn=<SelectBackward>)
tensor(0.3238, grad_fn=<SelectBackward>)
tensor(0.3065, grad_fn=<SelectBackward>)
tensor(0.3656, g

tensor(9.4625e-05, grad_fn=<SelectBackward>)
tensor(0.0002, grad_fn=<SelectBackward>)
tensor(0.7726, grad_fn=<SelectBackward>)
tensor(0.0934, grad_fn=<SelectBackward>)
tensor(0.9890, grad_fn=<SelectBackward>)
tensor(0.1671, grad_fn=<SelectBackward>)
tensor(0.4897, grad_fn=<SelectBackward>)
tensor(0.4352, grad_fn=<SelectBackward>)
tensor(0.2546, grad_fn=<SelectBackward>)
tensor(0.4462, grad_fn=<SelectBackward>)
tensor(0.3056, grad_fn=<SelectBackward>)
tensor(0.0934, grad_fn=<SelectBackward>)
tensor(0.9521, grad_fn=<SelectBackward>)
tensor(0.9940, grad_fn=<SelectBackward>)
tensor(0.5861, grad_fn=<SelectBackward>)
tensor(0.0140, grad_fn=<SelectBackward>)
tensor(6.8384e-06, grad_fn=<SelectBackward>)
tensor(2.6678e-05, grad_fn=<SelectBackward>)
tensor(0.0014, grad_fn=<SelectBackward>)
tensor(0.0414, grad_fn=<SelectBackward>)
tensor(0.0371, grad_fn=<SelectBackward>)
tensor(0.0350, grad_fn=<SelectBackward>)
tensor(0.0662, grad_fn=<SelectBackward>)
tensor(0.2491, grad_fn=<SelectBackward>)
tens

...

tensor(0.1134, grad_fn=<SelectBackward>)
tensor(0.0022, grad_fn=<SelectBackward>)
tensor(0.0016, grad_fn=<SelectBackward>)
tensor(0.0025, grad_fn=<SelectBackward>)
tensor(3.5513e-05, grad_fn=<SelectBackward>)
tensor(0.5173, grad_fn=<SelectBackward>)
tensor(0.0005, grad_fn=<SelectBackward>)
tensor(0.0005, grad_fn=<SelectBackward>)
tensor(0.0003, grad_fn=<SelectBackward>)
tensor(0.0244, grad_fn=<SelectBackward>)
tensor(0.7035, grad_fn=<SelectBackward>)
tensor(0.3778, grad_fn=<SelectBackward>)
tensor(0.7041, grad_fn=<SelectBackward>)
tensor(0.9626, grad_fn=<SelectBackward>)
tensor(0.7826, grad_fn=<SelectBackward>)
tensor(0.0079, grad_fn=<SelectBackward>)
tensor(0.0077, grad_fn=<SelectBackward>)
tensor(0.0010, grad_fn=<SelectBackward>)
tensor(0.2864, grad_fn=<SelectBackward>)
tensor(0.1546, grad_fn=<SelectBackward>)
tensor(1.7015e-05, grad_fn=<SelectBackward>)
tensor(1.3359e-05, grad_fn=<SelectBackward>)
tensor(0.7330, grad_fn=<SelectBackward>)
tensor(0.5963, grad_fn=<SelectBackward>)
tens

tensor(0.0083, grad_fn=<SelectBackward>)
tensor(0.0004, grad_fn=<SelectBackward>)
tensor(5.4792e-05, grad_fn=<SelectBackward>)
tensor(0.0634, grad_fn=<SelectBackward>)
tensor(0.0021, grad_fn=<SelectBackward>)
tensor(0.0020, grad_fn=<SelectBackward>)
tensor(0.0016, grad_fn=<SelectBackward>)
tensor(0.0003, grad_fn=<SelectBackward>)
tensor(0.0102, grad_fn=<SelectBackward>)
tensor(0.0009, grad_fn=<SelectBackward>)
tensor(0.0003, grad_fn=<SelectBackward>)
tensor(0.3832, grad_fn=<SelectBackward>)
tensor(0.0020, grad_fn=<SelectBackward>)
tensor(0.0008, grad_fn=<SelectBackward>)
tensor(0.0001, grad_fn=<SelectBackward>)
tensor(0.0005, grad_fn=<SelectBackward>)
tensor(0.0304, grad_fn=<SelectBackward>)
tensor(0.0131, grad_fn=<SelectBackward>)
tensor(0.0539, grad_fn=<SelectBackward>)
tensor(0.0220, grad_fn=<SelectBackward>)
tensor(6.4723e-05, grad_fn=<SelectBackward>)
tensor(0.0053, grad_fn=<SelectBackward>)
tensor(0.0207, grad_fn=<SelectBackward>)
tensor(0.0006, grad_fn=<SelectBackward>)
tensor(0

tensor(0.3855, grad_fn=<SelectBackward>)
tensor(0.0041, grad_fn=<SelectBackward>)
tensor(0.3745, grad_fn=<SelectBackward>)
tensor(0.1084, grad_fn=<SelectBackward>)
tensor(3.0814e-05, grad_fn=<SelectBackward>)
tensor(0.8868, grad_fn=<SelectBackward>)
tensor(0.0050, grad_fn=<SelectBackward>)
tensor(0.0036, grad_fn=<SelectBackward>)
tensor(0.0007, grad_fn=<SelectBackward>)
tensor(3.7200e-05, grad_fn=<SelectBackward>)
tensor(0.0007, grad_fn=<SelectBackward>)
tensor(0.0042, grad_fn=<SelectBackward>)
tensor(0.0046, grad_fn=<SelectBackward>)
tensor(0.0106, grad_fn=<SelectBackward>)
tensor(0.9506, grad_fn=<SelectBackward>)
tensor(0.0294, grad_fn=<SelectBackward>)
tensor(0.0970, grad_fn=<SelectBackward>)
tensor(0.6943, grad_fn=<SelectBackward>)
tensor(0.4408, grad_fn=<SelectBackward>)
tensor(0.3084, grad_fn=<SelectBackward>)
tensor(0.1302, grad_fn=<SelectBackward>)
tensor(0.0256, grad_fn=<SelectBackward>)
tensor(0.0146, grad_fn=<SelectBackward>)
tensor(0.0009, grad_fn=<SelectBackward>)
tensor(0

tensor(0.3005, grad_fn=<SelectBackward>)
tensor(0.0387, grad_fn=<SelectBackward>)
tensor(0.0093, grad_fn=<SelectBackward>)
tensor(0.0008, grad_fn=<SelectBackward>)
tensor(0.0004, grad_fn=<SelectBackward>)
tensor(0.0053, grad_fn=<SelectBackward>)
tensor(0.0011, grad_fn=<SelectBackward>)
tensor(0.0292, grad_fn=<SelectBackward>)
tensor(0.0008, grad_fn=<SelectBackward>)
tensor(0.0002, grad_fn=<SelectBackward>)
tensor(0.0042, grad_fn=<SelectBackward>)
tensor(0.0023, grad_fn=<SelectBackward>)
tensor(0.2469, grad_fn=<SelectBackward>)
tensor(0.4521, grad_fn=<SelectBackward>)
tensor(0.2245, grad_fn=<SelectBackward>)
tensor(0.4371, grad_fn=<SelectBackward>)
tensor(0.9643, grad_fn=<SelectBackward>)
tensor(0.0032, grad_fn=<SelectBackward>)
tensor(3.1235e-05, grad_fn=<SelectBackward>)
tensor(0.0107, grad_fn=<SelectBackward>)
tensor(0.0786, grad_fn=<SelectBackward>)
tensor(0.0344, grad_fn=<SelectBackward>)
tensor(0.0037, grad_fn=<SelectBackward>)
tensor(0.0082, grad_fn=<SelectBackward>)
tensor(5.571

tensor(1.7588e-06, grad_fn=<SelectBackward>)
tensor(0.0017, grad_fn=<SelectBackward>)
tensor(0.0014, grad_fn=<SelectBackward>)
tensor(0.0066, grad_fn=<SelectBackward>)
tensor(2.7490e-05, grad_fn=<SelectBackward>)
tensor(0.8646, grad_fn=<SelectBackward>)
tensor(0.0402, grad_fn=<SelectBackward>)
tensor(0.4932, grad_fn=<SelectBackward>)
tensor(0.1842, grad_fn=<SelectBackward>)
tensor(0.8340, grad_fn=<SelectBackward>)
tensor(0.5477, grad_fn=<SelectBackward>)
tensor(0.4810, grad_fn=<SelectBackward>)
tensor(0.2557, grad_fn=<SelectBackward>)
tensor(0.0042, grad_fn=<SelectBackward>)
tensor(0.0563, grad_fn=<SelectBackward>)
tensor(0.0006, grad_fn=<SelectBackward>)
tensor(1.8874e-05, grad_fn=<SelectBackward>)
tensor(0.0009, grad_fn=<SelectBackward>)
tensor(0.9893, grad_fn=<SelectBackward>)
tensor(0.1433, grad_fn=<SelectBackward>)
tensor(0.0467, grad_fn=<SelectBackward>)
tensor(0.0051, grad_fn=<SelectBackward>)
tensor(0.0005, grad_fn=<SelectBackward>)
tensor(0.0257, grad_fn=<SelectBackward>)
tens

In [23]:
cv2.destroyAllWindows()